In [1]:
#default_exp model.msms_param_search

In [2]:
#export
import torch
import alphadeep.model.msms as msms
import alphadeep.model.base as model_base

class ModelMSMSTest(torch.nn.Module):
    def __init__(self,
        mod_feature_size,
        num_ion_types,
        max_instrument_num,
        dropout=0.1,
        hidden_layer=2,
        hidden_nn = 'lstm',
        output_nn = 'lstm',
        cat_meta_first=True,
        output_transform = None,
    ):
        super().__init__()
        hidden_size=512

        self.cat_meta_first=cat_meta_first
        self.output_transform = output_transform

        self.meta_nn = msms.MetaNet(
            max_instrument_num, hidden_size
        )

        self.dropout = torch.nn.Dropout(dropout)

        self.input = msms.InputLSTM(
            mod_feature_size, hidden_size
        )

        if hidden_nn == 'transformer':
            self.hidden = msms.HiddenTransformer(
                hidden_size, dropout,
                hidden_layer
            )
        else:
            self.hidden = model_base.SeqLSTM(
                hidden_size, hidden_size,
                rnn_layer=hidden_layer, bidirectional=True
            )

        if output_nn == 'linear':
            self.output = torch.nn.Linear(
                hidden_size,
                num_ion_types,
                bias=False
            )
        else:
            self.output = model_base.SeqLSTM(
                hidden_size, num_ion_types,
                rnn_layer=1, bidirectional=False
            )

    def forward(self,
        aa_indices,
        mod_x,
        charges:torch.Tensor,
        NCEs:torch.Tensor,
        instrument_indices,
    ):
        x = self.input(aa_indices, mod_x)
        meta_x = self.meta_nn(charges, NCEs, instrument_indices)

        if self.cat_meta_first:
            x = x*meta_x.unsqueeze(1).repeat(1, x.size(1), 1)
        x = self.dropout(x)

        x = self.hidden(x)
        x = self.dropout(x)
        
        if not self.cat_meta_first:
            x = x*meta_x.unsqueeze(1).repeat(1, x.size(1), 1)

        x = self.output(x[:,3:,:])

        if self.output_transform == 'sigmoid':
            return torch.sigmoid(x)
        else:
            return x

class ParamSearchModel(msms.pDeepModel):
    def __init__(self,
        dropout=0.1,
        lr=0.001,
        model_class:torch.nn.Module=ModelMSMSTest,
        hidden_layer=2,
        hidden_nn = 'lstm',
        output_nn = 'lstm',
        cat_meta_first=True,
        output_transform = None,
        loss='L1', #or 'IA'
        IA_weight_pred = False
    ):
        super().__init__(
            dropout=dropout,
            lr = lr,
            model_class = model_class,
            hidden_layer=hidden_layer,
            hidden_nn = hidden_nn,
            output_nn = output_nn,
            cat_meta_first=cat_meta_first,
            output_transform=output_transform
        )
        if loss == 'L1':
            self.loss_func = torch.nn.L1Loss()
        else:
            self.loss_func = msms.IntenAwareLoss(weight_pred=IA_weight_pred)
        self.save_as = f'msms_h={hidden_nn}_o={output_nn}_mf={cat_meta_first}_'\
            f'ot={output_transform}_l={loss}'
        if loss != 'L1':
            self.save_as += f'_IAw={IA_weight_pred}'
        self.save_as += '.pth'

In [27]:
#export
import itertools
def product_dict(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        _dict = dict(zip(keys, instance))
        if _dict['IA_weight_pred']:
            if _dict['loss'] == 'L1' or not _dict['output_transform']:
                continue
        yield _dict
        

In [28]:
#export
def get_param_iter():
    return product_dict(
        hidden_nn = ['lstm','transformer'],
        output_nn = ['lstm','linear'],
        cat_meta_first=[True,False],
        output_transform = [None,'sigmoid'],
        loss=['L1','IA'],
        IA_weight_pred = [True,False],
    )

In [29]:
len(list(get_param_iter()))

40

In [30]:
next(get_param_iter())

{'hidden_nn': 'lstm',
 'output_nn': 'lstm',
 'cat_meta_first': True,
 'output_transform': None,
 'loss': 'L1',
 'IA_weight_pred': False}